In [45]:
import pickle
import numpy as np
import time
import pandas as pd
import yaml
import torch
import copy

import sys
import os
current_dir = os.path.dirname(os.getcwd())
sys.path.append(current_dir)

from master import MASTERModel

# Please install qlib first before load the data.
with open("../workflow_config_master_Alpha158.yaml", 'r') as f:
    config = yaml.safe_load(f)

universe = 'csi300'
prefix = 'opensource' # ['original','opensource'], which training data are you using
# universe = 'csi800'
# prefix = 'extend_1' # ['original','opensource'], which training data are you using
train_data_dir = f'../data/self_exp'
# print(f'{train_data_dir}/{prefix}/{universe}_dl_train.pkl')
# with open(f'{train_data_dir}/{prefix}/{universe}_extend_dl_train.pkl', 'rb') as f:
with open(f'{train_data_dir}/{prefix}/{universe}_self_dl_train.pkl', 'rb') as f:
    dl_train = pickle.load(f)

predict_data_dir = f'../data/self_exp/{prefix}'
# with open(f'{predict_data_dir}/{universe}_extend_dl_valid.pkl', 'rb') as f:
with open(f'{predict_data_dir}/{universe}_self_dl_valid.pkl', 'rb') as f:
    dl_valid = pickle.load(f)
# with open(f'{predict_data_dir}/{universe}_extend_dl_test.pkl', 'rb') as f:
with open(f'{predict_data_dir}/{universe}_self_dl_test.pkl', 'rb') as f:
    dl_test = pickle.load(f)
    
# test = pd.read_pickle(f'{predict_data_dir}/{universe}_dl_test.pkl')
# print(test.data)

print("Data Loaded.")


d_feat = 158
d_model = 256
t_nhead = 4
s_nhead = 2
dropout = 0.5
gate_input_start_index = 158
gate_input_end_index = 221

if universe == 'csi300':
    beta = 5
elif universe == 'csi500':
    beta = 3
elif universe == 'csi800':
    beta = 2
else:
    beta = 2

n_epoch = 90
lr = 1e-5
GPU = 0

Data Loaded.


In [52]:
import pickle
import torch

exp_ids = range(9)
epochs = range(10, 91, 10)
columns = ['IC', 'ICIR', 'RIC', 'RICIR']
multi_index = pd.MultiIndex.from_product([exp_ids, epochs], names=['exp_id', 'epoch'])

df = pd.DataFrame(index=multi_index, columns=columns, dtype=float)

for exp_id in exp_ids:
    for epoch in epochs:
        print(exp_id, epoch)
        model = MASTERModel(
            d_feat = d_feat, d_model = d_model, t_nhead = t_nhead, s_nhead = s_nhead, T_dropout_rate=dropout, S_dropout_rate=dropout,
            beta=beta, gate_input_end_index=gate_input_end_index, gate_input_start_index=gate_input_start_index,
            n_epochs=n_epoch, lr = lr, GPU = GPU, seed = None, train_stop_loss_thred = None,
            save_path='/dev/null', save_prefix=f'',
        )
        model.load_param(f'./model/csi300/xue_csi300_opensource_b8_e{exp_id}_{epoch}.pkl')
        pred, metrics = model.predict(dl_test)
        for k, v in metrics.items():
            df.loc[(exp_id, epoch), k] = v
        pred.to_csv(f'./predictions_xue/csi300/xue_csi300_opensource_b8_e{exp_id}_{epoch}.csv')


0 10
self.fitted: Previously trained.
0 20
self.fitted: Previously trained.
0 30
self.fitted: Previously trained.
0 40
self.fitted: Previously trained.
0 50
self.fitted: Previously trained.
0 60
self.fitted: Previously trained.
0 70
self.fitted: Previously trained.
0 80
self.fitted: Previously trained.
0 90
self.fitted: Previously trained.
1 10
self.fitted: Previously trained.
1 20
self.fitted: Previously trained.
1 30
self.fitted: Previously trained.
1 40
self.fitted: Previously trained.
1 50
self.fitted: Previously trained.
1 60
self.fitted: Previously trained.
1 70
self.fitted: Previously trained.
1 80
self.fitted: Previously trained.
1 90
self.fitted: Previously trained.
2 10
self.fitted: Previously trained.
2 20
self.fitted: Previously trained.
2 30
self.fitted: Previously trained.
2 40
self.fitted: Previously trained.
2 50
self.fitted: Previously trained.
2 60
self.fitted: Previously trained.
2 70
self.fitted: Previously trained.
2 80
self.fitted: Previously trained.
2 90
self.fi

In [48]:
df.loc[(0, [10, 20, 30, 40, 50, 60, 70, 80, 90]), :]

IC      ICIR       RIC     RICIR
exp_id epoch                                        
0      10     0.062078  0.325009  0.082418  0.441307
       20     0.057520  0.316514  0.071599  0.393575
       30     0.059005  0.313501  0.072073  0.385201
       40     0.057215  0.328240  0.064131  0.358145
       50     0.057082  0.312803  0.069627  0.376933
       60     0.055913  0.306559  0.068786  0.367811
       70     0.053356  0.289308  0.066177  0.347897
       80     0.047582  0.260306  0.057805  0.305323
       90     0.037951  0.207514  0.046359  0.246086

In [54]:
df.to_csv('./predictions_xue/exp_results.csv')